<a href="https://colab.research.google.com/github/starkjones/Project-3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import libraries:

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url ="https://datasets.imdbws.com/title.akas.tsv.gz"
rating_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"

# DataFrames: 

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(rating_url, sep='\t', low_memory=False)

basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
# Replacing null values:

basics.replace({'\\N':np.nan}, inplace =True) 
akas.replace({'\\N':np.nan}, inplace =True) 
ratings.replace({'\\N':np.nan}, inplace =True) 

ltering/Cleaning Steps:

Title Basics:

Replace "\N" with np.nan

Eliminate movies that are null for runtimeMinutes

Eliminate movies that are null for genre keep only titleType==Movie

keep startYear 2000-2022

Eliminate movies that include "Documentary" in genre (see tip below)

In [12]:
basics.info()

remove = basics['runtimeMinutes'] == '0'

remove

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9018777 entries, 0 to 9018776
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 619.3+ MB


0          False
1          False
2          False
3          False
4          False
           ...  
9018772    False
9018773    False
9018774    False
9018775    False
9018776    False
Name: runtimeMinutes, Length: 9018777, dtype: bool